In [ ]:
# transfer learning mobilenetv2 cat vs dog

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.preprocessing import image
from os import getcwd


In [ ]:
# Verileri web üzerinden indirin
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=url, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
# Dizin yolunu tanımlayın
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [ ]:
# Kediler ve köpekler için dizin yolunu tanımlayın
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
# Eğitim ve doğrulama dizinindeki kedi ve köpek görüntülerinin sayısını yazdıralım
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))
num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))
total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)
print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
--
Total training images: 2000
Total validation images: 1000


In [ ]:
# modelimizi tanımlayalım

# ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# doğrulama verilerimiz için ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(150, 150),  
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# Önceden eğitilmiş modeli indirelim.

mobile_net = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
mobile_net.trainable = False

In [ ]:
# Callback'imizi tanımlayalım.
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            print("% 99 doğruluğa ulaştı, bu nedenle eğitimi iptal ettim!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
mobile_net.trainable = False
mobile_net.summary()
x = layers.Flatten()(mobile_net.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = Model(mobile_net.input, x)

# modelimizi derleyelim.
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 75, 75, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 75, 75, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 75, 75, 32)   0           ['bn_Conv1[0][

In [ ]:
# modelimizi eğitelim.
history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=100, epochs=2, validation_steps=50, verbose=2, callbacks=[callbacks])


Epoch 1/2
100/100 - 87s - loss: 0.9764 - accuracy: 0.8450 - val_loss: 0.2151 - val_accuracy: 0.9390 - 87s/epoch - 868ms/step
Epoch 2/2
100/100 - 86s - loss: 0.4108 - accuracy: 0.8900 - val_loss: 0.2665 - val_accuracy: 0.9300 - 86s/epoch - 857ms/step


In [ ]:
CATS_VS_DOGS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)


In [ ]:
# artık elimizde eğitilmiş bir modelimiz var artık onu dönüştürebiliriz.
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'
with open(tflite_model_file, "wb") as f:
 f.write(tflite_model)


In [ ]:
# buraya kadar tamam. artık dönüştürdüğümüz model bizden nasıl bir input alacak
# ve nasıl bir output verecek ona bakalım.

interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
predictions = []

In [ ]:
# toplu tahmin ve sonuçlarını ekrana yazdıralım.

for img in os.listdir(validation_cats_dir):
    img = image.load_img(os.path.join(validation_cats_dir, img), target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    print(interpreter.get_tensor(output_index))

for img in os.listdir(validation_dogs_dir):
    img = image.load_img(os.path.join(validation_dogs_dir, img), target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    print(interpreter.get_tensor(output_index))

predictions = np.array(predictions)
predictions = predictions.reshape(-1)
predictions = np.round(predictions)

y_true = np.array([0] * num_cats_val + [1] * num_dogs_val)
y_pred = predictions

print(classification_report(y_true, y_pred))


In [ ]:
'''
test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
 interpreter.set_tensor(input_index, img)
 interpreter.invoke()
 predictions.append(interpreter.get_tensor(output_index))
 test_labels.append(label.numpy()[0])
 test_imgs.append(img)
'''

In [ ]:
'''
score = 0
for item in range(0,99):
 prediction=np.argmax(predictions[item])
 label = test_labels[item]
 if prediction==label:
  score=score+1
print("Aldığım 100 tahminden " + str(score) + " doğru")
'''

In [ ]:
'''
for index in range(0,99):
 plt.figure(figsize=(6,3))
 plt.subplot(1,2,1)
 plot_image(index, predictions, test_labels, test_imgs)
 plt.show()
'''

In [ ]:
url = 'https://i.natgeofe.com/n/5f35194b-af37-4f45-a14d-60925b280986/NationalGeographic_2731043_4x3.jpg'
path_to_img = tf.keras.utils.get_file('cat.1700.jpg', origin=url)

img = image.load_img(path_to_img, target_size=(150, 150))

x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)

def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

x = preprocess_input(x)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

print(classes[0])
if classes[0]>0.5:
    print(url + " resmi köpek !")
else:
    print(url + " resmi kedi '")

1/1 [==============================] - 0s 60ms/step
[1.]
https://i.natgeofe.com/n/5f35194b-af37-4f45-a14d-60925b280986/NationalGeographic_2731043_4x3.jpg resmi köpek !


In [ ]:
'''
# modelimizi optimize edelim.

converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_data_gen():
 for input_value, _ in test_batches.take(100):
  yield [input_value]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'
with open(tflite_model_file, "wb") as f:
 f.write(tflite_model)
'''
